The pytorch based implementation produces unexpected results. We will use this, and its
 counterpart in the other implementation, to try to debug the issue.


In [22]:
import numpy as np
import tensorflow as tf
import gpflow
%matplotlib
import matplotlib.pyplot as plt

from examples.utils.plotting import *
from gpflow.config import default_float
from gpflow_sampling.samplers import decoupled
from gpflow_sampling.samplers.decoupled_samplers import _decoupled_sampler_gpr


Using matplotlib backend: Qt5Agg


In [23]:
# generate the train data
train_X = tf.reshape(tf.range(0, 1, 0.1, dtype=default_float()), shape=[-1, 1])
train_Y = train_X * tf.sin(20*train_X)

# initialize the GP model
model = gpflow.models.GPR(
    data=(train_X, train_Y),
    kernel=gpflow.kernels.Matern52(lengthscales=0.1, variance=1.0),
    noise_variance=5e-3
)

# test points
test_X = tf.reshape(tf.range(0, 1, 0.01, dtype=default_float()), shape=[-1, 1])

# clear previous plots
plt.close()

# posterior summary
# model.predict_f() returns f_mean and f_variance
# predict_y() returns the same, except with added observation noise
post_mean, post_var = model.predict_y(test_X)
post_mean = tf.reshape(post_mean, shape=[-1])
post_std = tf.reshape(tf.sqrt(post_var), shape=[-1])
plt.plot(tf.reshape(test_X, shape=[-1]), post_mean, label="post_mean", color="green")
plt.fill_between(
    tf.reshape(test_X, shape=[-1]),
    post_mean - 2 * post_std,
    post_mean + 2 * post_std,
    alpha=0.5
)

# plot several posterior samples - use predict_f_samples
num_samples = 10
exact_samples = tf.squeeze(
    model.predict_f_samples(test_X, num_samples=num_samples),
    axis=-1
)
plt.plot(
    tf.reshape(test_X, shape=[-1]),
    tf.transpose(exact_samples),
    label="exact",
    color="blue"
)


In [24]:
# initialize the decoupled sampler
ds = _decoupled_sampler_gpr(
    model=model,
    kernel=None,  # unused kwarg
    sample_shape=[num_samples],
    num_basis=256
)

# sample from decoupled
ds_samples = ds(test_X)

# plot the samples
plt.plot(
    tf.reshape(test_X, [-1]),
    tf.transpose(tf.squeeze(ds_samples, -1)),
    label="decoupled",
    color="red"
)


In [25]:
# plot
plt.grid(True)
plt.legend()
plt.title("GPflow")
plt.show()